# BranchPro: inference of R_t with Poisson Binomial Noise

The first part of the notebook includes a forward simulation of the incidence numbers for an example branching process model with Poisson Binomial noise.

The second part of the notebook focuses on the computation of the posterior of the reproduction number for the inference using the data from the previous section, using two methods:
- using the posterior class implementation.

The mean and 95% interval quantiles are plotted together to illustrate a sensible trajectory of the R profile in time for both methods of inference.

In [1]:
# Import libraries
import numpy as np
import math
import branchpro
import scipy.stats
from branchpro.apps import ReproductionNumberPlot
import plotly.graph_objects as go
import pandas as pd
from cmdstanpy import CmdStanModel,  cmdstan_path
import arviz as az
import nest_asyncio
nest_asyncio.apply()

num_timepoints = 100 # number of days for incidence data


## Parameterize example branching process model

In [2]:
# Build the next generation time distribution theta_s
ws_mean = 8
ws_std = 2
w_dist = scipy.stats.norm(ws_mean, scale=ws_std).pdf(np.arange(1, 21))
disc_w = 0.02 * w_dist / np.max(w_dist)

# Simulate incidence data
initial_mu = 3 * 7
next_gen = disc_w
m = branchpro.PoiBinBranchProModel(initial_mu, next_gen)
new_mus = [21, 14, 7]
start_times = [0, 20, 40]
m.set_mean_contact(new_mus, start_times)
parameters = 10 # initial number of cases
times = np.arange(num_timepoints)
 
cases = m.simulate(parameters, times)
print(cases)

[10.  0.  0.  0.  0.  1.  4.  2.  3.  3.  5.  2.  4.  3.  2.  6.  5.  8.
  4.  4.  2.  7.  3.  7.  6.  3. 10.  5.  5. 11.  7.  9. 12.  3. 10. 12.
 10. 13. 11. 12. 11.  8.  8.  7.  7.  7.  6. 11.  6. 10.  3.  3.  3.  5.
  5.  6.  4.  2.  5.  3.  2.  2.  3.  2.  0.  5.  1.  1.  2.  1.  3.  2.
  0.  0.  1.  0.  2.  3.  2.  1.  0.  0.  2.  1.  1.  1.  0.  0.  2.  0.
  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.]


## Plot local incidence numbers

In [3]:
# Plot (bar chart cases each day)
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=cases,
        name='Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()

## Compute the posterior distribution using STAN

In [12]:
poibin_model = """
functions {
    // Return the Poisson-binomial log probability mass for the specified
    // count y and vector of probabilities theta.  The result is the log
    // probability of y successes in N = size(theta) independent
    // trials with probabilities of success theta[1], ..., theta[N].
    //
    // See:  https://en.wikipedia.org/wiki/Poisson_binomial_distribution
    //
    // @param y number of successes
    // @param theta vector of probabilities
    // @return Poisson-binomial log probability mass
    //
    real poisson_binomial_lpmf(int y, vector theta) {
        int N = rows(theta);
        matrix[N + 1, N + 1] alpha;
        vector[N] log_theta = log(theta);
        vector[N] log1m_theta = log1m(theta);

        if (y < 0 || y > N)
        reject("poisson binomial variate y out of range, y = ", y,
                " N = ", N);
        for (n in 1:N)
        if (theta[n] < 0 || theta[n] > 1)
            reject("poisson binomial parameter out of range,",
                " theta[", n, "] =", theta[n]);

        if (N == 0)
        return y == 0 ? 0 : negative_infinity();

        // dynamic programming with cells
        // alpha[n + 1, tot + 1] = log prob of tot successes in first n trials
        alpha[1, 1] = 0;
        for (n in 1:N) {
        // tot = 0
        alpha[n + 1, 1] = alpha[n, 1] + log1m_theta[n];

        // 0 < tot < n
        for (tot in 1:min(y, n - 1))
            alpha[n + 1, tot + 1]
                = log_sum_exp(alpha[n, tot] + log_theta[n],
                            alpha[n, tot  + 1] + log1m_theta[n]);

        // tot = n
        if (n > y) continue;
        alpha[n + 1, n + 1] = alpha[n, n] + log_theta[n];
        }
        return alpha[N + 1, y + 1];
    }
    int [] evaluate_previous_cases (int t, array [] int aI, array [] real aMu) {
        array [t-1] int rev_prev_cases;
        int counter;

        for (k in 1:(t-1)) {
            rev_prev_cases[k] = 0;
            counter = 0;
            while (counter < floor(aMu[t])) {
                rev_prev_cases[k] += aI[t-k];
                counter += 1;
            }
        }
        return rev_prev_cases;
    }
    vector evaluate_poissonbinomial_probabilities (
        int N, int S, int t, array [] int aI, array [] real aTheta, array [] real aMu) {
        array [t-1] int rev_prev_cases;
        int ci;
        
        rev_prev_cases = evaluate_previous_cases(t, aI, aMu);
        
        vector [sum(rev_prev_cases)] pp;

        pp[:rev_prev_cases[1]] = rep_vector(aTheta[1], rev_prev_cases[1]);
        ci = rev_prev_cases[1];
        for(k in 2:(t-1)) {
            if(k <= S) {
                pp[(ci + 1):(ci + rev_prev_cases[k])] = rep_vector(
                    aTheta[k], rev_prev_cases[k]);
            }
            else {
                pp[(ci + 1):(ci + rev_prev_cases[k])] = rep_vector(
                    0, rev_prev_cases[k]);
            }
        }
        return pp;
    }
}
data {
    int N; // number of days
    int S; // length probability distribution
    array [N] int I; // local incidences for N days
    array [S] real Theta; // probability distribution of
    // that the infector infects a contact s days after they
    // first displays symptoms.
}
parameters {
    array [N] real<lower=0, upper=50> Mu;
    // vector of time-dependent mean number of contacts
}
model {
    for(t in 2:N) {
        if (sum(evaluate_previous_cases(t, I, Mu)) != 0){
            target += poisson_binomial_lpmf(
                I[t] |
                evaluate_poissonbinomial_probabilities(
                    N, S, t, I, Theta, Mu)
                ); // likelihood
        }
    }
    for(t in 1:N){
        Mu[t] ~ gamma(14, 0.5);
    }
}
"""

poibin_data = {
    'N': num_timepoints,
    'S': len(next_gen),
    'I': cases.astype(np.integer).tolist(),
    'Theta': next_gen.tolist()}

/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/ipykernel_1569/3794640659.py:119: DeprecationWarning:

Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.



In [15]:
posterior = stan.build(
    poibin_model, data=poibin_data, random_seed=10)

fit = posterior.sample(
    num_chains=3, num_samples=10
    )

samples = az.from_cmdstanpy(
    fit,
    observed_data='I',
    coords={'observation': list(range(num_timepoints)),
            'covariate': [
                '{}'.format(_)
                for _ in range(num_timepoints)]
            },
    dims={
        'I': ['observation'],
        'Mu': ['covariate']
        })

az.summary(samples)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 50, column 4: Declaration
    of arrays by placing brackets after a type is deprecated and will be
    removed in Stan 2.33.0. Instead use the array keyword before the type.
    This can be changed automatically using the auto-format flag to stanc
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 111, column 22: Argument
    14 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 102, column 8: A
    control flow statement depends on parameter(s): Mu.
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 76, column 12: A
    control flow statement

In [ ]:
az.rcParams['plot.max_subplots'] = 2 * num_timepoints

az.plot_trace(
    samples,
    var_names=('Mu'),
    filter_vars='like',
    compact=False)

In [ ]:
# Eliminate burn-in iterations (1/2 of the chain lengths)
chain_samples = fit._draws[7:, 500:, :]

# Evaluate the model for all parameter sets in the samples
n_param, n_sample, n_chains = chain_samples.shape

extended_samples = np.concatenate((
    chain_samples[:, :, 0],
    chain_samples[:, :, 1],
    chain_samples[:, :, 2]), axis=1)

thinning = max(1, int(n_sample * n_chains / 50))

new_intervals = pd.DataFrame({
    'Time Points': np.arange(R_t_start, num_timepoints),
    'Mean': np.mean(extended_samples[:, ::thinning], axis=1),
    'Lower bound CI': np.quantile(extended_samples[:, ::thinning], 0.025, axis=1),
    'Upper bound CI': np.quantile(extended_samples[:, ::thinning], 0.975, axis=1),
    'Central Probability': (L1+L2) * [0.95]
})

## Inference plot using class method results

In [ ]:
fig = ReproductionNumberPlot()

fig.add_ground_truth_rt(ground_truth)
fig.add_interval_rt(new_intervals)

fig.update_labels(time_label='Time (Day)', r_label='R_t')

fig.show_figure()

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

